## 카카오 로컬 API 활용
- 도로명 주소 --> 위도, 경도 좌표

In [1]:
import requests, json
import pandas as pd 
from urllib.parse import quote

- API Key

In [2]:
with open('keys/카카오apiKey.txt') as file:
    kakao_key = file.read()

- URL 만들기

In [3]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 영등포구 경인로 지하843 (영등포동3가)'
url = f'{base_url}?query={quote(addr)}'

In [4]:
# Header: Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {kakao_key}'}

- Kakao Local API 호출하여 결과 가져오기

In [5]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 영등포구 영등포동3가 33',
    'b_code': '1156010400',
    'h_code': '1156053500',
    'main_address_no': '33',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_h_name': '영등포동',
    'region_3depth_name': '영등포동3가',
    'sub_address_no': '',
    'x': '126.906658750671',
    'y': '37.5164625580995'},
   'address_name': '서울 영등포구 경인로 지하 843',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 영등포구 경인로 지하 843',
    'building_name': '영등포역지하도상가',
    'main_building_no': '843',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_name': '영등포동3가',
    'road_name': '경인로',
    'sub_building_no': '',
    'underground_yn': 'Y',
    'x': '126.906658750671',
    'y': '37.5164625580995',
    'zone_no': '07305'},
   'x': '126.906658750671',
   'y': '37.5164625580995'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [6]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [7]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.5164625580995, 126.906658750671)

- 영등포소재 지하철역 정보 완성하기

In [8]:
df = pd.read_csv('data/영등포소재 지하철역.csv')
df

,이름,주소
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가)
1,신도림역,서울특별시 구로구 경인로 625 (신도림동)
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가)
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가)


In [9]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [10]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,이름,주소,위도,경도
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가),37.516463,126.906659
1,신도림역,서울특별시 구로구 경인로 625 (신도림동),37.507084,126.884719
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가),37.534957,126.902731
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.521842,126.924485
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가),37.525700,126.896627


In [11]:
df.to_csv('data/영등포소재 지하철역2.csv', index=False)
pd.read_csv('data/영등포소재 지하철역2.csv')

,이름,주소,위도,경도
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가),37.516463,126.906659
1,신도림역,서울특별시 구로구 경인로 625 (신도림동),37.507084,126.884719
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가),37.534957,126.902731
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.521842,126.924485
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가),37.525700,126.896627
